In [ ]:
from flask import Flask, request
from nornir import InitNornir
import pynetbox
#from nornir.core.inventory import Host
from ipaddress import IPv4Address
from ipaddress import IPv4Network
import ipaddress
from jinja2 import Environment, FileSystemLoader

In [ ]:
nr = InitNornir(
    inventory={
        "plugin": "NetBoxInventory2",
        "options": {
            "nb_url": "http://10.30.1.101:8000",
            "nb_token": "dc5b766afda60e647a9fcc384c736a9689da8618",
            "group_file": "./inventory/groups.yml",
            "defaults_file": "./inventory/defaults.yml",
        },
    },
)

In [ ]:
nb_url = nr.config.inventory.options['nb_url']
nb_token = nr.config.inventory.options['nb_token']
nb = pynetbox.api(
    nb_url,
    token=nb_token
)

In [ ]:
nb_site = 'plgn'
get_all_devices = list(nb.dcim.devices.filter(site = nb_site))
#get_all_devices

In [ ]:
# List all IPs prefixes from all sites
nb_site = 'plgn'
nb_role = 'plgn'
IPv4 = 4
get_all_ip_prefixes = list(nb.ipam.prefixes.filter(site = nb_site, role = nb_role))
for ip_prefix in get_all_ip_prefixes:
    if ip_prefix.family.value == IPv4:
        print(ip_prefix.description)
        print(ip_prefix)

In [ ]:
# List all IPs from all devices
get_all_ip_addresses = list(nb.ipam.ip_addresses.filter(device=get_all_devices))
#get_all_ip_addresses

In [ ]:
# List all interfaces from all devices
get_all_interfaces = list(nb.dcim.interfaces.filter(device=get_all_devices))
#get_all_interfaces

In [ ]:

templates_path = "./templates/"
environment = Environment(loader=FileSystemLoader(templates_path))
template = environment.get_template("dhcpd_static.template")
for device_ip in get_all_ip_addresses:
        for ip_prefix in get_all_ip_prefixes:
                if IPv4Address(device_ip.address.split('/')[0]) in IPv4Network(ip_prefix.prefix):
                        for device_interface in get_all_interfaces:
                                if device_ip.assigned_object.device.name == device_interface.device.name and device_interface.mac_address != None:
                                        content = template.render(
                                        device_name = device_interface.device.name,
                                        mac_address = device_interface.mac_address,
                                        ip_address = device_ip.address.split('/')[0]
                                        )
                                        print(content)

In [6]:
# Create a Flask instance
app = Flask(__name__)

@app.route('/api/fixed_ip', methods=['POST'])
def webhook():
    """ get_device_ips = [request.json["data"]["address"]]
    get_device = request.json["data"]["assigned_object"]["device"]["name"]
    #get_interface = request.json["data"]["assigned_object"]["name"]
    get_address_family = request.json["data"]["family"]["value"]
    #get_assigned_object_id = request.json["data"]["assigned_object_id"]
    get_device_interface = nb.dcim.interfaces.get(request.json["data"]["assigned_object_id"])
    #print(get_device_ips,device)

    templates_path = "./templates/"
    environment = Environment(loader=FileSystemLoader(templates_path))
    template = environment.get_template("dhcpd_static.template")
    #device_interface = list(nb.dcim.interfaces.filter(device=get_device, name=get_interface))

    if get_device_interface.mgmt_only:
            print("\tManagement interface, no changes will be performed...")
    else:
        for device_ip in get_device_ips:
            #print(ipaddress.ip_interface(device_ip).ip)
            content = template.render(
            device_name = get_device,
            mac_address = get_device_interface.mac_address,
            ip_address = ipaddress.ip_interface(device_ip).ip
            )
            print(content) """

    print(request.json)
    
    return "Webhook received!"

app.run(host='0.0.0.0', port=8080)
if __name__ == "__main__": app.run()


10.30.1.101 - - [13/Mar/2023 14:52:45] "POST /api/fixed_ip HTTP/1.1" 200 -


{'event': 'updated', 'timestamp': '2023-03-13 09:52:44.283343+00:00', 'model': 'ipaddress', 'username': 'admin', 'request_id': '83d6d31f-98e6-4249-bffb-66e7c6fc0119', 'data': {'id': 110, 'url': '/api/ipam/ip-addresses/110/', 'display': '1.1.1.1/24', 'family': {'value': 4, 'label': 'IPv4'}, 'address': '1.1.1.1/24', 'vrf': None, 'tenant': None, 'status': {'value': 'dhcp', 'label': 'DHCP'}, 'role': None, 'assigned_object_type': 'dcim.interface', 'assigned_object_id': 59, 'assigned_object': {'id': 59, 'url': '/api/dcim/interfaces/59/', 'display': 'Gig-E 3', 'device': {'id': 8, 'url': '/api/dcim/devices/8/', 'display': 'SRV-PEV-1', 'name': 'SRV-PEV-1'}, 'name': 'Gig-E 3', 'cable': None, '_occupied': False}, 'nat_inside': None, 'nat_outside': [], 'dns_name': '', 'description': '', 'comments': '', 'tags': [], 'custom_fields': {}, 'created': '2023-03-13T06:56:08.474289Z', 'last_updated': '2023-03-13T09:52:44.139155Z'}, 'snapshots': {'prechange': {'created': '2023-03-13T06:56:08.474Z', 'last_up

 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [ ]:
ip = ['10.30.1.101/24']
interface = 'iDRAC'
device = 'SRV-PEV-1'
templates_path = "./templates/"
environment = Environment(loader=FileSystemLoader(templates_path))
template = environment.get_template("dhcpd_static.template")
device_interface = list(nb.dcim.interfaces.filter(device=device, name=interface))

assigned_interface_details = nb.dcim.interfaces.get('61')
print(assigned_interface_details)
for device_ip in ip:
    content = template.render(
    device_name = device,
    mac_address = device_interface[0].mac_address,
    ip_address = ipaddress.ip_interface(device_ip).ip
    )
    print(content)